In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon July 16 10:08:53 2018

@author: sxx
"""
import time
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import traceback 

from predict import load_data as load_data
from predict import transform_data as transform_data
from predict import format_data as format_data
from predict import divide_data as divide_data
from predict import LstmModel as LstmModel

Using TensorFlow backend.
/opt/anaconda3/envs/health/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
current_time=time.strftime('%Y-%m-%d-%H-%M',time.localtime())

def experiments(params):
    print ("epoch:",params['epoch']," batch_size: ",params['batch'])
    csvfile = './data/health_1799.csv'
    x,y = load_data(csvfile)
    x_,y_ = transform_data(x,y,False)
    datax,datay = format_data(x_,y_)
    trainX,trainY,validX,validY,testX,testY = divide_data(datax,datay)
    
    try:
        ##调试模型epoch,batch_size,learnrate
        #model = LstmModel(epoches=params['epoch'], batch_size=params['batch'],lr=params['lr'])

        ##测试模型units,dropout_rate
        model = LstmModel(units=params['units'],dr=params['dr'],lr=0.01,epoches=10,batch_size=4)
        health_model = model.create_model()
        save_path = './models/health_{0}.h5'.format(current_time)
        model.train_model_1(trainX,trainY,validX,validY,save_path)
        testPredict = health_model.predict(testX)
        print (testY,"-->",testPredict)
    except Exception as e:
        print ("something happen:",repr(e))
        print ('-' * 20)
        with open("./data/err.log","a") as logf:
            traceback.print_exc(file=logf)
        return {'loss': 999999, 'status': STATUS_OK}
    
    mse = np.mean(np.square(testPredict - testY))
    print ("mse:",mse)
    print ('-' * 20)

    return {'loss': mse, 'status': STATUS_OK}


In [3]:
if __name__ == '__main__':
    space = {
        'units':{
        'ls_units1': hp.choice('ls_units1',[i for i in range(12,16,4)]),
        'ls_units2': hp.choice('ls_units2',[i for i in range(8,12,4)]),
        'ls_units3': hp.choice('ls_units3',[i for i in range(8,12,4)]),
        'ls_units4': hp.choice('ls_units4',[i for i in range(4,8,4)]),      
        'ds_units5': hp.choice('ds_units5',[i for i in range(6,8,2)])},
        'dr': hp.uniform('dr',0.1,0.6),
        'batch': hp.randint('batch',256),
        'epoch': hp.randint('epoch',500),
        'lr': hp.uniform('lr',0.0001,0.01),
        'activation': hp.choice('activation',['relu',
                                                'sigmoid',
                                                'tanh',
                                                'linear'])
        }
    
    trials = Trials()
    best = fmin(experiments, space, algo=tpe.suggest, max_evals=5, trials=trials)
    print ('best: ',best)
    print ("hyperoptparam end")

epoch: 444  batch_size:  198
LSTM model unit1_size:12 unit2_size:8 unit3_size:8 unit4_size:4 unit5_size:6 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 15, 12)            3120      
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 12)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 12)            48        
_________________________________________________________________
lstm_2 (LSTM)                (None, 15, 8)             672       
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 8)             0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 8)             32        
__________________________________


Epoch 00008: val_loss did not improve from 0.00384
Epoch 9/10
69/69 [==============================] - 1s 10ms/step - loss: 0.0044 - val_loss: 0.0037

Epoch 00009: val_loss improved from 0.00384 to 0.00367, saving model to ./models/health_checkpoint_2018-08-03-13-14.hdf5
Epoch 10/10
69/69 [==============================] - 1s 10ms/step - loss: 0.0036 - val_loss: 0.0036

Epoch 00010: val_loss improved from 0.00367 to 0.00356, saving model to ./models/health_checkpoint_2018-08-03-13-14.hdf5
[[0.975 0.96  0.97  0.975 0.96 ]
 [0.96  0.97  0.975 0.96  0.96 ]
 [0.97  0.975 0.96  0.96  0.95 ]
 [0.975 0.96  0.96  0.95  0.68 ]
 [0.96  0.96  0.95  0.68  0.97 ]
 [0.96  0.95  0.68  0.97  0.95 ]
 [0.95  0.68  0.97  0.95  0.96 ]
 [0.68  0.97  0.95  0.96  0.965]
 [0.97  0.95  0.96  0.965 0.97 ]
 [0.95  0.96  0.965 0.97  0.97 ]] --> [[0.9380464  0.9272213  0.9358179  0.9341223  0.93022287]
 [0.9378503  0.9288119  0.9350772  0.93469864 0.9304348 ]
 [0.9371567  0.9298199  0.9346899  0.9348177  0.931943


Epoch 00003: val_loss did not improve from 0.00356
Epoch 4/10
69/69 [==============================] - 1s 10ms/step - loss: 0.0518 - val_loss: 0.0203

Epoch 00004: val_loss did not improve from 0.00356
Epoch 5/10
69/69 [==============================] - 1s 10ms/step - loss: 0.0218 - val_loss: 0.0117

Epoch 00005: val_loss did not improve from 0.00356
Epoch 6/10
69/69 [==============================] - 1s 10ms/step - loss: 0.0088 - val_loss: 0.0080

Epoch 00006: val_loss did not improve from 0.00356
Epoch 7/10
69/69 [==============================] - 1s 10ms/step - loss: 0.0082 - val_loss: 0.0060

Epoch 00007: val_loss did not improve from 0.00356
Epoch 8/10
69/69 [==============================] - 1s 10ms/step - loss: 0.0058 - val_loss: 0.0113

Epoch 00008: val_loss did not improve from 0.00356
Epoch 9/10
69/69 [==============================] - 1s 10ms/step - loss: 0.0049 - val_loss: 0.0078

Epoch 00009: val_loss did not improve from 0.00356
Epoch 00009: early stopping
[[0.975 0.96  

In [5]:
import hyperopt
print (hyperopt.space_eval(space, best))

{'activation': 'linear', 'batch': 180, 'dr': 0.21504192960744692, 'epoch': 92, 'lr': 0.00934276179215912, 'units': {'ds_units5': 6, 'ls_units1': 12, 'ls_units2': 8, 'ls_units3': 8, 'ls_units4': 4}}
